# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars[:100]

array([33, 78, 61, 76, 30, 17, 42, 25, 64,  2,  2,  2, 54, 61, 76, 76, 82,
       25,  1, 61, 12, 26, 65, 26, 17, 19, 25, 61, 42, 17, 25, 61, 65, 65,
       25, 61, 65, 26, 69, 17, 43, 25, 17, 48, 17, 42, 82, 25, 16, 10, 78,
       61, 76, 76, 82, 25,  1, 61, 12, 26, 65, 82, 25, 26, 19, 25, 16, 10,
       78, 61, 76, 76, 82, 25, 26, 10, 25, 26, 30, 19, 25, 13, 27, 10,  2,
       27, 61, 82,  6,  2,  2, 49, 48, 17, 42, 82, 30, 78, 26, 10],
      dtype=int32)

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the virst split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:,:10]

array([[33, 78, 61, 76, 30, 17, 42, 25, 64,  2],
       [ 3, 10, 22, 25, 78, 17, 25, 12, 13, 48],
       [25, 24, 61, 30, 24, 78, 26, 10, 35, 25],
       [13, 30, 78, 17, 42, 25, 27, 13, 16, 65],
       [25, 30, 78, 17, 25, 65, 61, 10, 22, 18],
       [25, 11, 78, 42, 13, 16, 35, 78, 25, 65],
       [30, 25, 30, 13,  2, 22, 13,  6,  2,  2],
       [13, 25, 78, 17, 42, 19, 17, 65,  1, 41],
       [78, 61, 30, 25, 26, 19, 25, 30, 78, 17],
       [17, 42, 19, 17, 65,  1, 25, 61, 10, 22]], dtype=int32)

I'll write another function to grab batches out of the arrays made by split data. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [15]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
        
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')
    
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # Build the RNN layers
    with tf.name_scope("RNN_cells"):
        def make_cell():
            lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
            return drop
        cell = tf.contrib.rnn.MultiRNNCell([make_cell() for _ in range(num_layers)])
    
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    
    final_state = state
    
    # Reshape output so it's a bunch of rows, one row for each cell output
    with tf.name_scope('sequence_reshape'):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN outputs to a softmax layer and calculate the cost
    with tf.name_scope('logits'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b
        tf.summary.histogram('softmax_w', softmax_w)
        tf.summary.histogram('softmax_b', softmax_b)

    with tf.name_scope('predictions'):
        preds = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', preds)
    
    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        tf.summary.scalar('cost', cost)

    # Optimizer for training, using gradient clipping to control exploding gradients
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    merged = tf.summary.merge_all()
    
    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer', 'merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. The two you probably haven't seen before are `lstm_size` and `num_layers`. These set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but you'll have to watch out for overfitting. If your validation loss is much larger than the training loss, you're probably overfitting. Decrease the size of the network or decrease the dropout keep probability.

In [11]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

## Training

Time for training which is is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

In [12]:
!mkdir -p checkpoints/anna

In [13]:
def train(model, epochs, file_writer):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Use the line below to load a checkpoint and resume training
        #saver.restore(sess, 'checkpoints/anna20.ckpt')

        n_batches = int(train_x.shape[1]/num_steps)
        iterations = n_batches * epochs
        for e in range(epochs):

            # Train network
            new_state = sess.run(model.initial_state)
            loss = 0
            for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
                iteration = e*n_batches + b
                start = time.time()
                feed = {model.inputs: x,
                        model.targets: y,
                        model.keep_prob: 0.5,
                        model.initial_state: new_state}
                summary, batch_loss, new_state, _ = sess.run([model.merged, model.cost, 
                                                              model.final_state, model.optimizer], 
                                                              feed_dict=feed)
                loss += batch_loss
                end = time.time()
                print('Epoch {}/{} '.format(e+1, epochs),
                      'Iteration {}/{}'.format(iteration, iterations),
                      'Training loss: {:.4f}'.format(loss/b),
                      '{:.4f} sec/batch'.format((end-start)))

                file_writer.add_summary(summary, iteration)

In [16]:
epochs = 20
batch_size = 100
num_steps = 100
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

for lstm_size in [128,256,512]:
    for num_layers in [1, 2]:
        for learning_rate in [0.002, 0.001]:
            log_string = 'logs/4/lr={},rl={},ru={}'.format(learning_rate, num_layers, lstm_size)
            writer = tf.summary.FileWriter(log_string)
            model = build_rnn(len(vocab), 
                    batch_size=batch_size,
                    num_steps=num_steps,
                    learning_rate=learning_rate,
                    lstm_size=lstm_size,
                    num_layers=num_layers)
            
            train(model, epochs, writer)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch 1/20  Iteration 1/3560 Training loss: 4.4115 0.3018 sec/batch
Epoch 1/20  Iteration 2/3560 Training loss: 4.3983 0.2493 sec/batch
Epoch 1/20  Iteration 3/3560 Training loss: 4.3830 0.2709 sec/batch
Epoch 1/20  Iteration 4/3560 Training loss: 4.3606 0.2037 sec/batch
Epoch 1/20  Iteration 5/3560 Training loss: 4.3165 0.2074 sec/batch
Epoch 1/20  Iteration 6/3560 Training loss: 4.2289 0.2081 sec/batch
Epoch 1/20  Iteration 7/3560 Training loss: 4.1369 0.1856 sec/batch
Epoch 1/20  Iteration 8/3560 Training loss: 4.0583 0.2239 sec/batch
Epoch 1/20  Iteration 9/3560 Training loss: 3.9904 0.1963 sec/batch
Epoch 1/20  Iteration 10/3560 Training loss: 3.9320 0.1859 sec/batch
Epoch 1/20  Iteration 11/3560 Training loss: 3.8779 0.1732 sec/batch
Epoch 1/20  Iteration 12/3560 Training loss: 3.8321 0.2024

Epoch 1/20  Iteration 115/3560 Training loss: 3.1492 0.2348 sec/batch
Epoch 1/20  Iteration 116/3560 Training loss: 3.1463 0.2496 sec/batch
Epoch 1/20  Iteration 117/3560 Training loss: 3.1435 0.1803 sec/batch
Epoch 1/20  Iteration 118/3560 Training loss: 3.1408 0.1833 sec/batch
Epoch 1/20  Iteration 119/3560 Training loss: 3.1383 0.1929 sec/batch
Epoch 1/20  Iteration 120/3560 Training loss: 3.1355 0.1932 sec/batch
Epoch 1/20  Iteration 121/3560 Training loss: 3.1329 0.1761 sec/batch
Epoch 1/20  Iteration 122/3560 Training loss: 3.1303 0.2051 sec/batch
Epoch 1/20  Iteration 123/3560 Training loss: 3.1275 0.2065 sec/batch
Epoch 1/20  Iteration 124/3560 Training loss: 3.1249 0.1967 sec/batch
Epoch 1/20  Iteration 125/3560 Training loss: 3.1221 0.1830 sec/batch
Epoch 1/20  Iteration 126/3560 Training loss: 3.1192 0.1767 sec/batch
Epoch 1/20  Iteration 127/3560 Training loss: 3.1165 0.1995 sec/batch
Epoch 1/20  Iteration 128/3560 Training loss: 3.1138 0.1904 sec/batch
Epoch 1/20  Iteratio

Epoch 2/20  Iteration 234/3560 Training loss: 2.4746 0.1850 sec/batch
Epoch 2/20  Iteration 235/3560 Training loss: 2.4738 0.1785 sec/batch
Epoch 2/20  Iteration 236/3560 Training loss: 2.4726 0.1809 sec/batch
Epoch 2/20  Iteration 237/3560 Training loss: 2.4716 0.1810 sec/batch
Epoch 2/20  Iteration 238/3560 Training loss: 2.4709 0.1866 sec/batch
Epoch 2/20  Iteration 239/3560 Training loss: 2.4698 0.1815 sec/batch
Epoch 2/20  Iteration 240/3560 Training loss: 2.4691 0.1895 sec/batch
Epoch 2/20  Iteration 241/3560 Training loss: 2.4684 0.1759 sec/batch
Epoch 2/20  Iteration 242/3560 Training loss: 2.4673 0.1922 sec/batch
Epoch 2/20  Iteration 243/3560 Training loss: 2.4662 0.1906 sec/batch
Epoch 2/20  Iteration 244/3560 Training loss: 2.4656 0.2037 sec/batch
Epoch 2/20  Iteration 245/3560 Training loss: 2.4648 0.2015 sec/batch
Epoch 2/20  Iteration 246/3560 Training loss: 2.4634 0.1954 sec/batch
Epoch 2/20  Iteration 247/3560 Training loss: 2.4623 0.1945 sec/batch
Epoch 2/20  Iteratio

Epoch 2/20  Iteration 352/3560 Training loss: 2.3920 0.2177 sec/batch
Epoch 2/20  Iteration 353/3560 Training loss: 2.3917 0.2001 sec/batch
Epoch 2/20  Iteration 354/3560 Training loss: 2.3912 0.1971 sec/batch
Epoch 2/20  Iteration 355/3560 Training loss: 2.3906 0.2133 sec/batch
Epoch 2/20  Iteration 356/3560 Training loss: 2.3900 0.1902 sec/batch
Epoch 3/20  Iteration 357/3560 Training loss: 2.3390 0.1817 sec/batch
Epoch 3/20  Iteration 358/3560 Training loss: 2.2990 0.1786 sec/batch
Epoch 3/20  Iteration 359/3560 Training loss: 2.2893 0.1848 sec/batch
Epoch 3/20  Iteration 360/3560 Training loss: 2.2872 0.1819 sec/batch
Epoch 3/20  Iteration 361/3560 Training loss: 2.2858 0.1961 sec/batch
Epoch 3/20  Iteration 362/3560 Training loss: 2.2839 0.1890 sec/batch
Epoch 3/20  Iteration 363/3560 Training loss: 2.2846 0.2031 sec/batch
Epoch 3/20  Iteration 364/3560 Training loss: 2.2861 0.1859 sec/batch
Epoch 3/20  Iteration 365/3560 Training loss: 2.2875 0.1838 sec/batch
Epoch 3/20  Iteratio

Epoch 3/20  Iteration 471/3560 Training loss: 2.2449 0.1818 sec/batch
Epoch 3/20  Iteration 472/3560 Training loss: 2.2442 0.1763 sec/batch
Epoch 3/20  Iteration 473/3560 Training loss: 2.2438 0.2145 sec/batch
Epoch 3/20  Iteration 474/3560 Training loss: 2.2433 0.1805 sec/batch
Epoch 3/20  Iteration 475/3560 Training loss: 2.2431 0.1833 sec/batch
Epoch 3/20  Iteration 476/3560 Training loss: 2.2428 0.1757 sec/batch
Epoch 3/20  Iteration 477/3560 Training loss: 2.2426 0.1827 sec/batch
Epoch 3/20  Iteration 478/3560 Training loss: 2.2421 0.1774 sec/batch
Epoch 3/20  Iteration 479/3560 Training loss: 2.2417 0.1799 sec/batch
Epoch 3/20  Iteration 480/3560 Training loss: 2.2415 0.1758 sec/batch
Epoch 3/20  Iteration 481/3560 Training loss: 2.2411 0.1794 sec/batch
Epoch 3/20  Iteration 482/3560 Training loss: 2.2405 0.1794 sec/batch
Epoch 3/20  Iteration 483/3560 Training loss: 2.2402 0.1791 sec/batch
Epoch 3/20  Iteration 484/3560 Training loss: 2.2400 0.1915 sec/batch
Epoch 3/20  Iteratio

...

Epoch 17/20  Iteration 2996/3560 Training loss: 1.2209 3.0795 sec/batch
Epoch 17/20  Iteration 2997/3560 Training loss: 1.2210 3.1367 sec/batch
Epoch 17/20  Iteration 2998/3560 Training loss: 1.2209 3.2466 sec/batch
Epoch 17/20  Iteration 2999/3560 Training loss: 1.2206 3.1209 sec/batch
Epoch 17/20  Iteration 3000/3560 Training loss: 1.2204 3.1522 sec/batch
Epoch 17/20  Iteration 3001/3560 Training loss: 1.2204 3.1895 sec/batch
Epoch 17/20  Iteration 3002/3560 Training loss: 1.2204 3.1498 sec/batch
Epoch 17/20  Iteration 3003/3560 Training loss: 1.2204 3.1365 sec/batch
Epoch 17/20  Iteration 3004/3560 Training loss: 1.2203 3.1356 sec/batch
Epoch 17/20  Iteration 3005/3560 Training loss: 1.2203 3.3085 sec/batch
Epoch 17/20  Iteration 3006/3560 Training loss: 1.2203 3.1523 sec/batch
Epoch 17/20  Iteration 3007/3560 Training loss: 1.2200 3.0899 sec/batch
Epoch 17/20  Iteration 3008/3560 Training loss: 1.2201 3.2539 sec/batch
Epoch 17/20  Iteration 3009/3560 Training loss: 1.2203 3.1476 se

Epoch 18/20  Iteration 3110/3560 Training loss: 1.2137 3.2014 sec/batch
Epoch 18/20  Iteration 3111/3560 Training loss: 1.2135 3.1233 sec/batch
Epoch 18/20  Iteration 3112/3560 Training loss: 1.2134 3.1158 sec/batch
Epoch 18/20  Iteration 3113/3560 Training loss: 1.2133 3.2969 sec/batch
Epoch 18/20  Iteration 3114/3560 Training loss: 1.2129 3.0899 sec/batch
Epoch 18/20  Iteration 3115/3560 Training loss: 1.2125 3.0523 sec/batch
Epoch 18/20  Iteration 3116/3560 Training loss: 1.2126 3.3284 sec/batch
Epoch 18/20  Iteration 3117/3560 Training loss: 1.2123 3.0551 sec/batch
Epoch 18/20  Iteration 3118/3560 Training loss: 1.2123 3.1491 sec/batch
Epoch 18/20  Iteration 3119/3560 Training loss: 1.2120 3.2284 sec/batch
Epoch 18/20  Iteration 3120/3560 Training loss: 1.2117 3.1585 sec/batch
Epoch 18/20  Iteration 3121/3560 Training loss: 1.2114 3.1171 sec/batch
Epoch 18/20  Iteration 3122/3560 Training loss: 1.2114 3.1726 sec/batch
Epoch 18/20  Iteration 3123/3560 Training loss: 1.2114 3.2660 se

Epoch 19/20  Iteration 3224/3560 Training loss: 1.2083 3.0804 sec/batch
Epoch 19/20  Iteration 3225/3560 Training loss: 1.2079 3.1739 sec/batch
Epoch 19/20  Iteration 3226/3560 Training loss: 1.2083 3.1716 sec/batch
Epoch 19/20  Iteration 3227/3560 Training loss: 1.2075 3.1246 sec/batch
Epoch 19/20  Iteration 3228/3560 Training loss: 1.2077 3.1199 sec/batch
Epoch 19/20  Iteration 3229/3560 Training loss: 1.2077 3.1827 sec/batch
Epoch 19/20  Iteration 3230/3560 Training loss: 1.2063 3.2983 sec/batch
Epoch 19/20  Iteration 3231/3560 Training loss: 1.2054 3.1961 sec/batch
Epoch 19/20  Iteration 3232/3560 Training loss: 1.2059 3.1742 sec/batch
Epoch 19/20  Iteration 3233/3560 Training loss: 1.2059 3.1531 sec/batch
Epoch 19/20  Iteration 3234/3560 Training loss: 1.2060 3.0858 sec/batch
Epoch 19/20  Iteration 3235/3560 Training loss: 1.2052 3.1129 sec/batch
Epoch 19/20  Iteration 3236/3560 Training loss: 1.2043 3.1961 sec/batch
Epoch 19/20  Iteration 3237/3560 Training loss: 1.2041 3.1553 se

Epoch 19/20  Iteration 3338/3560 Training loss: 1.1946 3.1753 sec/batch
Epoch 19/20  Iteration 3339/3560 Training loss: 1.1946 3.2118 sec/batch
Epoch 19/20  Iteration 3340/3560 Training loss: 1.1947 3.1074 sec/batch
Epoch 19/20  Iteration 3341/3560 Training loss: 1.1950 3.1322 sec/batch
Epoch 19/20  Iteration 3342/3560 Training loss: 1.1950 3.2094 sec/batch
Epoch 19/20  Iteration 3343/3560 Training loss: 1.1951 3.0958 sec/batch
Epoch 19/20  Iteration 3344/3560 Training loss: 1.1952 3.2965 sec/batch
Epoch 19/20  Iteration 3345/3560 Training loss: 1.1955 3.1102 sec/batch
Epoch 19/20  Iteration 3346/3560 Training loss: 1.1957 3.1730 sec/batch
Epoch 19/20  Iteration 3347/3560 Training loss: 1.1956 3.2003 sec/batch
Epoch 19/20  Iteration 3348/3560 Training loss: 1.1959 3.1721 sec/batch
Epoch 19/20  Iteration 3349/3560 Training loss: 1.1958 3.2610 sec/batch
Epoch 19/20  Iteration 3350/3560 Training loss: 1.1960 3.1343 sec/batch
Epoch 19/20  Iteration 3351/3560 Training loss: 1.1960 3.1739 se

Epoch 20/20  Iteration 3452/3560 Training loss: 1.1927 3.1643 sec/batch
Epoch 20/20  Iteration 3453/3560 Training loss: 1.1934 3.1615 sec/batch
Epoch 20/20  Iteration 3454/3560 Training loss: 1.1935 3.1730 sec/batch
Epoch 20/20  Iteration 3455/3560 Training loss: 1.1940 3.1669 sec/batch
Epoch 20/20  Iteration 3456/3560 Training loss: 1.1937 3.1455 sec/batch
Epoch 20/20  Iteration 3457/3560 Training loss: 1.1937 3.0180 sec/batch
Epoch 20/20  Iteration 3458/3560 Training loss: 1.1939 3.2509 sec/batch
Epoch 20/20  Iteration 3459/3560 Training loss: 1.1938 3.3928 sec/batch
Epoch 20/20  Iteration 3460/3560 Training loss: 1.1937 3.0358 sec/batch
Epoch 20/20  Iteration 3461/3560 Training loss: 1.1930 3.1918 sec/batch
Epoch 20/20  Iteration 3462/3560 Training loss: 1.1929 3.1829 sec/batch
Epoch 20/20  Iteration 3463/3560 Training loss: 1.1924 3.1761 sec/batch
Epoch 20/20  Iteration 3464/3560 Training loss: 1.1923 3.2138 sec/batch
Epoch 20/20  Iteration 3465/3560 Training loss: 1.1917 3.1320 se

In [17]:
tf.train.get_checkpoint_state('checkpoints/anna')

model_checkpoint_path: "checkpoints/anna/i1780_l512_1.256.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i200_l512_2.436.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i400_l512_1.992.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i600_l512_1.758.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i800_l512_1.604.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1000_l512_1.485.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1200_l512_1.407.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1400_l512_1.348.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1600_l512_1.306.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1780_l512_1.256.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [18]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [19]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [21]:
checkpoint = "checkpoints/anna/i3560_l512_1.122.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/anna/i3560_l512_1.122.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/anna/i3560_l512_1.122.ckpt
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "/Users/xlnwel/anaconda3/envs/ai/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/xlnwel/anaconda3/envs/ai/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/xlnwel/anaconda3/envs/ai/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/xlnwel/anaconda3/envs/ai/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()

In [ ]:
checkpoint = "checkpoints/anna/i200_l512_2.432.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = "checkpoints/anna/i600_l512_1.750.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = "checkpoints/anna/i1000_l512_1.484.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)